In [ ]:
from multiprocessing import cpu_count, Pool
from sklearn import *
import numpy as np
import pandas as pd
import random
import glob

sdir = '/kaggle/input/AI4Code/'
train = glob.glob(sdir+'/train/**') #139256
test = glob.glob(sdir+'/test/**') #4 / 20K code comp
sub = pd.read_csv(sdir+'sample_submission.csv') #id, cell_order

In [ ]:
%%time
def getDF(files):
    dfs = []
    for f in files:
        d = eval(open(f, 'r').read())
        df = pd.DataFrame([[c, r, d['cell_type'][c], d['source'][c]] for r, c in enumerate(d['cell_type'])], columns=['cell_id','rank','cell_type','source'])
        df['id'] = f.split('/')[-1].split('.')[0]
        dfs.append(df)
    return pd.concat(dfs)

def multi_transform(a):
    p = Pool(cpu_count())
    df = p.map(getDF, np.array_split(a, cpu_count()))
    df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
    p.close(); p.join()
    return df

train = multi_transform(train[:1000]) #example only
test = multi_transform(test)
print(train.shape, test.shape)
train.head()

In [ ]:
#sharding networks for nlp processing
print(train[train['rank']==0]['cell_type'].value_counts())
end = train.drop_duplicates(subset=['id'], keep='last')
end = end[end['cell_type']=='markdown']
print(end.shape)

In [ ]:
sub = []
for f in test['id'].unique():
    c = list(test[((test['id']==f) & (test['cell_type']=='code'))].sort_values('rank')['cell_id'].values)
    m = list(test[((test['id']==f) & (test['cell_type']=='markdown'))].sort_values('rank')['cell_id'].values)
    random.shuffle(m)
    o = c[:int(len(c)/2)] + m[:] + c[int(len(c)/2):]
    sub.append([f, ' '.join(o)])
sub = pd.DataFrame(sub, columns=['id', 'cell_order'])
sub.to_csv('submission.csv', index=False)